In [1]:
import numpy as np
from utils.generator import generate_positive_definite_matrix, make_quadratic_function
from itertools import product
from scipy.optimize import minimize
from tqdm import tqdm

In [2]:
n = 15

In [3]:
Q = generate_positive_definite_matrix(n)
c = np.random.normal(size=n)
mu0 = 1.5 / np.min(np.linalg.eigvalsh(Q))

In [4]:
def brute_force(Q, c):
    n = len(c)
    obj_fn = make_quadratic_function(Q, c)
    all_possible_x = np.array(list(product([0, 1], repeat=n)))
    x_star = all_possible_x[np.argmin(np.array(list(map(obj_fn, all_possible_x))))]
    return x_star

In [5]:
def binary_admm(obj_fn, x0, y0, mu0, n_iter=100, gamma=0.9):
    n = len(x0)
    x = z = x0
    y = y0
    mu = mu0
    bnds = [(0,1)] * n
    for _ in range(n_iter):
        lx = lambda x: obj_fn(x) + np.dot(y, x - z) + 1/(2*mu) * (np.linalg.norm(x - z) ** 2)
        res = minimize(lx, x, bounds=bnds)
        x = res.x
        z = np.where(x ** 2 / (2 * mu) < (x - 1) ** 2 / (2 * mu) - y, 0, 1)
        y += (x - z) / mu
        mu *= gamma
    return x, y

In [6]:
correct = 0
n_iter = 100
for i in tqdm(range(n_iter)):
    Q = generate_positive_definite_matrix(n)
    c = np.random.normal(size=n)
    k = np.min(np.linalg.eigvalsh(Q))
    obj_fn = make_quadratic_function(Q, c)
    mu = 1/k * 1.5
    xhat, yhat = binary_admm(obj_fn, np.ones(n) * 0.5, np.zeros(n), mu, 100, gamma=0.9)
    xhat = np.where(xhat > 0.5, 1, 0)
    xstar = brute_force(Q, c)
    if np.sum(xhat ^ xstar):
        print(mu, np.sum(xhat ^ xstar), np.linalg.cond(Q))
    correct += 1 - np.sum(xhat ^ xstar) / n

print(correct / n_iter)

  0%|          | 0/100 [00:00<?, ?it/s]

  9%|▉         | 9/100 [00:04<00:42,  2.16it/s]

116.79536899722052 2 356.0707401283331


 10%|█         | 10/100 [00:04<00:41,  2.15it/s]

148.1648545296715 2 587.1003252967416


...

137.0764474173562 4 477.63460227908433


 94%|█████████▍| 94/100 [00:42<00:02,  2.12it/s]

64.19619894006634 2 197.80052718475008


100%|██████████| 100/100 [00:45<00:00,  2.22it/s]

0.9813333333333333
